In [1]:
import pandas as pd
from pymongo import MongoClient

pd.set_option('display.max_colwidth', -1)
pd.options.display.float_format = '${:,.2f}'.format

def _connect_mongo(uri, db):
    """ A util for making a connection to mongo """

    if uri:
#         mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(uri)
    else:
        conn = MongoClient(host, port)
    return conn[db]

def read_mongo(uri, db, collection, query={}, no_id=True, limit=None):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    con = _connect_mongo(uri, db)

    # Make a query to the specific DB and Collection
    ### SET THE LIMIT TO 5 FOR DEV AND TESTING ###
    if limit:
        cursor = con[collection].find(query)
        cursor = cursor[limit]
    else:
        cursor = con[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df = pd.DataFrame(list(cursor))
    # Delete the _id
#     if no_id:
#         del df['_id']
    if limit:
        print(f'number of indexes created has been limited to {limit} ..........................')

    return df


/Users/mastacow/.local/share/virtualenvs/forecast-forecast-GZGA6O4g/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [2]:
import config
import overalls
import weather
import benedict

In [3]:
w = weather.Weather('27606', 'observation')

AttributeError: 'dict' object has no attribute 'iteritems'

In [4]:
print(w.weather)

{'_id': 'DEFAULT', 'clouds': 'DEFAULT', 'rain': {'1h': 0, '3h': 0}, 'snow': {'1h': 0, '3h': 0}, 'wind': {'speed': 0, 'deg': 0}, 'humidity': 'DEFAULT', 'pressure': {'press': 'DEFAULT', 'sea_level': 'DEFAULT'}, 'temperature': {'temp': 'DEFAULT', 'temp_max': 'DEFAULT', 'temp_min': 'DEFAULT'}, 'status': 'DEFAULT', 'detailed_status': 'DEFAULT', 'weather_code': 'DEFAULT', 'visibility_distance': 0, 'dewpoint': 'DEFAULT', 'humidex': 'DEFAULT', 'heat_index': 'DEFAULT', 'time_to_instant': 'DEFAULT'}


In [5]:
collection = 'legit_inst'
db = 'owmap'
df = read_mongo(config.uri, db, collection, no_id=False, limit=5)

number of indexes created has been limited to 5 ..........................


In [6]:
def read_mongo_a(uri, db, collection, limit=None):
    ''' Retrieve data from the Mongo database and transform it to a pandas
    DataFrame; return the DataFrame.

    :param collection: the collection name
    :type collection: string
    :param limit: optional limiter to the number of documents retrieved. '''
    
    database = _connect_mongo(config.uri, db)
    col = {}
    
    # Shorten the cursor length if limit is given, otherwise get everything;
    # transform the retrieved data to a pandas.DataFrame and return it.
    if limit:
        cursor = database[collection].find({})
        cursor = cursor
        for doc in cursor[:limit]:
            col[doc['_id']] = doc
        # Transform the data collection to a dataframe.
        df = pd.DataFrame.from_dict(col, orient='index')
        print(f'The length of your df has been limited to {limit}.')
        return df
    else:
        cursor = database[collection].find({})
        for doc in cursor:
            col[doc['_id']] = doc
        # Transform the data collection to a dataframe.
        df = pd.DataFrame.from_dict(col, orient='index')
        return df
    
    # Let the user know that even though there were no errors the DataFrame
    # was not created.
    print('From read_mongo(): There were no errors, but your dataframe was not \
    created.')

def errors(casts, obs):
    ''' Make a dict of errors for the forecasts. Any dicts in the arguments
    will be flattened before comparison.
    
    :param casts: a list of dictionaries
    :param obs: a dictionary
    
    * For best results all dicts should have all the same keys and subkeys.
    '''
    
    # Flatten all dicts and compare. Add the comparisons to a list and return.
    casts = [overalls.flatten_dict(cast) for cast in casts]
    obs = overalls.flatten_dict(obs)
    return [overalls.compare_dicts(cast, obs) for cast in casts]

def gen_errs_df(df):
    ''' Create an errors dataframe from the argument.
    
    :param df: Must be a pandas DataFrame.
    '''
    ### is there a way to step through three lists together? ###
    errs_list = []
    errs_dict = {}
    # Create the error dicts list to be added to the errs_dict.
    for (obs, casts) in zip(df['weather'], df['forecasts']): ### this creates a the list of errors from each instant
        errs_list.append(errors(casts, obs))
#     df['errs'] = errs
#     print(df.head())
    for (_id, errs) in zip(df['_id'], errs_list):  ### this creates a dict from the errors list with the index as key
        errs_dict[_id] = errs
#     errs_dict
    dd = pd.DataFrame.from_dict(errs_dict, orient='index') ### this puts that dict into DataFrame form
#     print(dd.head())
    for c in dd.columns:  ### this replaces the errors DataFrame dictionaries with a list of their values 
        dd[c] = [list(d.values()) for d in dd[c]]
    return dd
